# Evaluation scores



example: <br>
higher log probability ==> predicted class
if [score 0]=-2 and [score 1]=-3 then it means that the predicted class is class 0 <br>
[score] = -2+3 = 1 > 0 --> class 0
<br>
###########
<br>
if [score 0]=-3 and [score 1]=-2 then it means that the predicted class is class 1 <br>
[score] = -3+2 = -1 < 0 --> class 1

***
## Load the scores

In [1]:
import csv
import re
import sys

import matplotlib.pyplot as plt
import pandas as pd
from src.utils import *
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, auc

In [2]:
probs_rawnet_clean_csv = '../eval/prob_rawnet_eval.csv'
probs_rawnet_clean = pd.read_csv(probs_rawnet_clean_csv, header=0, engine='python')

probs_rawnet_0dot2_csv = '../eval/prob_rawnet_eval_FGSM_0dot2.csv'
probs_rawnet_0dot2 = pd.read_csv(probs_rawnet_0dot2_csv, header=0, engine='python')

probs_rawnet_1dot0_csv = '../eval/prob_rawnet_eval_FGSM_1dot0.csv'
probs_rawnet_1dot0 = pd.read_csv(probs_rawnet_1dot0_csv, header=0, engine='python')

probs_rawnet_2dot0_csv = '../eval/prob_rawnet_eval_FGSM_2dot0.csv'
probs_rawnet_2dot0 = pd.read_csv(probs_rawnet_2dot0_csv, header=0, engine='python')

probs_rawnet_3dot0_csv = '../eval/prob_rawnet_eval_FGSM_3dot0.csv'
probs_rawnet_3dot0 = pd.read_csv(probs_rawnet_3dot0_csv, header=0, engine='python')


In [3]:
# paths to csv score files
probs_clean_csv = '../eval/prob_resnet_spec_eval.csv'
probs_0dot0_csv = '../eval/prob_resnet_spec_eval_FGSM_0dot0.csv'
probs_0dot2_csv = '../eval/prob_resnet_spec_eval_FGSM_0dot2.csv'
probs_0dot4_csv = '../eval/prob_resnet_spec_eval_FGSM_0dot4.csv'
probs_0dot6_csv = '../eval/prob_resnet_spec_eval_FGSM_0dot8.csv'
probs_0dot8_csv = '../eval/prob_resnet_spec_eval_FGSM_0dot8.csv'
probs_1dot0_csv = '../eval/prob_resnet_spec_eval_FGSM_1dot0.csv'
probs_2dot0_csv = '../eval/prob_resnet_spec_eval_FGSM_2dot0.csv' 
probs_3dot0_csv = '../eval/prob_resnet_spec_eval_FGSM_3dot0.csv'


# check the lengths
probs_clean = pd.read_csv(probs_clean_csv, header=0, engine='python')
probs_0dot0 = pd.read_csv(probs_0dot0_csv, header=0, engine='python')
probs_0dot2 = pd.read_csv(probs_0dot2_csv, header=0, engine='python')
probs_0dot4 = pd.read_csv(probs_0dot4_csv, header=0, engine='python')
probs_0dot6 = pd.read_csv(probs_0dot8_csv, header=0, engine='python')
probs_0dot8 = pd.read_csv(probs_0dot8_csv, header=0, engine='python')
probs_1dot0 = pd.read_csv(probs_1dot0_csv, header=0, engine='python')
probs_2dot0 = pd.read_csv(probs_2dot0_csv, header=0, engine='python')
probs_3dot0 = pd.read_csv(probs_3dot0_csv, header=0, engine='python')



if len(probs_clean) == len(probs_0dot2) == len(probs_0dot4) == len(probs_0dot6) == len(probs_0dot8) == len(probs_1dot0) == len(probs_2dot0) == len(probs_3dot0):
    print(f'Lengths are okay: {len(probs_clean)-1}')
else:
    sys.exit('Lengths are not okay')

SystemExit: Lengths are not okay

/nas/home/wwang/miniconda3/envs/thesis/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
len(probs_clean)

64

In [ ]:
# ground truth file of evaluation dataset (ASVSpoof2019)
config_path = '../config/residualnet_train_config.yaml'
config = read_yaml(config_path)
df_eval = pd.read_csv(os.path.join('..', config['df_eval_path']))

In [ ]:
df_eval[:3]

***
## Extract PREDICTED labels in the same order given by df_eval_19!

In [ ]:
def extract_id(file_path):
    match = re.search(r'LA_E_(\d+)', file_path)
    if match:
        return match.group(1)
    return None
    
    
def pred_probabilities(file2_path):
    # read df_eval_19
    file1_path = '../data/df_eval_19.csv'
    
    file1_ids = []
    with open(file1_path, 'r') as file1:
        csv_reader = csv.reader(file1)
        for row in csv_reader:
            file_id = extract_id(row[1])
            if file_id:
                file1_ids.append(file_id)
    
    # read second file and store data in a dictionary
    file2_data = {}
    with open(file2_path, 'r') as file2:
        csv_reader = csv.reader(file2)
        for row in csv_reader:
            file_id = extract_id(row[0])
            if file_id:
                file2_data[file_id] = (float(row[1]), float(row[2]))
                
    output_array = []
    for file_id in file1_ids:
        if file_id in file2_data:
            col2, col3 = file2_data[file_id]
            output_array.append(0 if col2>col3 else 1)
                
    return output_array

***
## Accuracies

In [ ]:
# the score values .csv uses a space as a delimiter between the file path and the score

# def convert_probs_to_labels(csv_file):
#     binary_list = []
#     
#     with open(csv_file, 'r') as file:
#         reader = csv.reader(file)
#         next(reader) # skip the header
#         for line in reader:
#             val_1 = float(line[1])
#             val_2 = float(line[2])
#             
#             try:
#                 if val_1 > val_2:
#                     binary_list.append(0)
#                 else:
#                     binary_list.append(1)
#             except ValueError:
#                 pass
#         
#     return binary_list

In [ ]:
pred_labels_clean_rawnet = pred_probabilities(file2_path=probs_rawnet_clean_csv)
pred_labels_0dot2_rawnet = pred_probabilities(file2_path=probs_rawnet_0dot2_csv)
pred_labels_1dot0_rawnet = pred_probabilities(file2_path=probs_rawnet_1dot0_csv)
pred_labels_2dot0_rawnet = pred_probabilities(file2_path=probs_rawnet_2dot0_csv)
pred_labels_3dot0_rawnet = pred_probabilities(file2_path=probs_rawnet_3dot0_csv)

In [ ]:
pred_labels_clean = pred_probabilities(file2_path=probs_clean_csv)
pred_labels_0dot0 = pred_probabilities(file2_path=probs_0dot0_csv)
pred_labels_0dot2 = pred_probabilities(file2_path=probs_0dot2_csv)
pred_labels_0dot4 = pred_probabilities(file2_path=probs_0dot4_csv)
pred_labels_0dot6 = pred_probabilities(file2_path=probs_0dot6_csv)
pred_labels_0dot8 = pred_probabilities(file2_path=probs_0dot8_csv)
pred_labels_1dot0 = pred_probabilities(file2_path=probs_1dot0_csv)
pred_labels_2dot0 = pred_probabilities(file2_path=probs_2dot0_csv)
pred_labels_3dot0 = pred_probabilities(file2_path=probs_3dot0_csv)

In [ ]:
pred_labels_clean[:10]

In [ ]:
# get the GT labels
GT_labels = df_eval.iloc[:, -1].tolist()

### Unbalanced accuracy

In [ ]:
unb_acc_rawnet_clean = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_rawnet)
unb_acc_rawnet_0dot2 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot2_rawnet)
unb_acc_rawnet_1dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_1dot0_rawnet)
unb_acc_rawnet_2dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_2dot0_rawnet)
unb_acc_rawnet_3dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_3dot0_rawnet)

print(f'The unbalanced acc for clean set for RawNet2 is {unb_acc_rawnet_clean*100:.2f}%\n'
      f'The unbalanced acc for eps=0.2 set for RawNet2 is {unb_acc_rawnet_0dot2*100:.2f}%\n'
      f'The unbalanced acc for eps=1.0 set for RawNet2 is {unb_acc_rawnet_1dot0*100:.2f}%\n'
      f'The unbalanced acc for eps=2.0 set for RawNet2 is {unb_acc_rawnet_2dot0*100:.2f}%\n'
      f'The unbalanced acc for eps=3.0 set for RawNet2 is {unb_acc_rawnet_3dot0*100:.2f}%\n')

In [ ]:
unb_acc_clean = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean)
unb_acc_0dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot0)
unb_acc_0dot2 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot2)
unb_acc_0dot4 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot4)
unb_acc_0dot6 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot8)
unb_acc_0dot8 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot8)
unb_acc_1dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_1dot0)
unb_acc_2dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_2dot0)
unb_acc_3dot0 = accuracy_score(y_true=GT_labels, y_pred=pred_labels_3dot0)

print(f'The unbalanced acc for clean eval set is {unb_acc_clean*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 0.2 set is {unb_acc_0dot2*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 0.4 set is {unb_acc_0dot4*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 0.6 set is {unb_acc_0dot6*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 0.8 set is {unb_acc_0dot8*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 1.0 set is {unb_acc_1dot0*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 2.0 set is {unb_acc_2dot0*100:.2f}%\n'
      f'The unbalanced acc for epsilon = 3.0 set is {unb_acc_3dot0*100:.2f}%')


In [ ]:
unb_acc_0dot0

### Balanced accuracy

In [ ]:
bal_acc_clean_rawnet = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_rawnet)
bal_acc_0dot2_rawnet = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot2_rawnet)
bal_acc_1dot0_rawnet = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_1dot0_rawnet)
bal_acc_2dot0_rawnet = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_2dot0_rawnet)
bal_acc_3dot0_rawnet = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_3dot0_rawnet)


print(f'The balanced acc for clean set for RawNet2 is {bal_acc_clean_rawnet*100:.2f}%\n'
      f'The balanced acc for epsilon = 0.2 set for RawNet2 is {bal_acc_0dot2_rawnet*100:.2f}%\n'
      f'The balanced acc for epsilon = 1.0 set for RawNet2 is {bal_acc_1dot0_rawnet*100:.2f}%\n'
      f'The balanced acc for epsilon = 2.0 set for RawNet2 is {bal_acc_2dot0_rawnet*100:.2f}%\n'
      f'The balanced acc for epsilon = 3.0 set for RawNet2 is {bal_acc_3dot0_rawnet*100:.2f}%\n')

In [ ]:
bal_acc_clean = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean)
bal_acc_0dot0 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot0)
bal_acc_0dot2 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot2)
bal_acc_0dot4 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot4)
bal_acc_0dot6 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot6)
bal_acc_0dot8 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_0dot8)
bal_acc_1dot0 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_1dot0)
bal_acc_2dot0 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_2dot0)
bal_acc_3dot0 = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_3dot0)

print(f'The balanced acc for clean eval set is {bal_acc_clean*100:.2f}%\n'
      f'The balanced acc for epsilon = 0.2 set is {bal_acc_0dot2*100:.2f}%\n'
      f'The balanced acc for epsilon = 0.4 set is {bal_acc_0dot4*100:.2f}%\n'
      f'The balanced acc for epsilon = 0.6 set is {bal_acc_0dot6*100:.2f}%\n'
      f'The balanced acc for epsilon = 0.8 set is {bal_acc_0dot8*100:.2f}%\n'
      f'The balanced acc for epsilon = 1.0 set is {bal_acc_1dot0*100:.2f}%\n'
      f'The balanced acc for epsilon = 2.0 set is {bal_acc_2dot0*100:.2f}%\n'
      f'The balanced acc for epsilon = 3.0 set is {bal_acc_3dot0*100:.2f}%')

In [ ]:
bal_acc_0dot0

***
## ROC curve and AUC

In [ ]:
def create_prob_pos_class_list(file2_path):
    # read df_eval_19 and get a list of the file names
    file1_path = '../data/df_eval_19.csv'
    
    file1_ids = []
    with open(file1_path, 'r') as file1:
        csv_reader = csv.reader(file1)
        for row in csv_reader:
            file_id = extract_id(row[1])
            if file_id:
                file1_ids.append(file_id)
    
    # read second file and store data in a dictionary
    file2_data = {}
    with open(file2_path, 'r') as file2:
        csv_reader = csv.reader(file2)
        for row in csv_reader:
            file_id = extract_id(row[0])
            if file_id:
                file2_data[file_id] = (float(row[1]), float(row[2]))
                
    output_array = []
    for file_id in file1_ids:
        if file_id in file2_data:
            col2, col3 = file2_data[file_id]
            output_array.append(col3) # get the probability value
                
    return output_array

# def create_prob_pos_class_list(csv_file):
#     probs_pos_class = []
#     with open(csv_file, 'r') as file:
#         reader = csv.reader(file)
#         next(reader) # skip the header
#         for row in reader:
#             val = float(row[2]) # negative
#             probs_pos_class.append(val)
#     return probs_pos_class

In [ ]:
positive_class_probs = create_prob_pos_class_list(probs_clean_csv)
positive_class_probs_0dot2 = create_prob_pos_class_list(probs_0dot2_csv)
positive_class_probs_0dot4 = create_prob_pos_class_list(probs_0dot4_csv)
positive_class_probs_0dot6 = create_prob_pos_class_list(probs_0dot6_csv)
positive_class_probs_0dot8 = create_prob_pos_class_list(probs_0dot8_csv)
positive_class_probs_1dot0 = create_prob_pos_class_list(probs_1dot0_csv)
positive_class_probs_2dot0 = create_prob_pos_class_list(probs_2dot0_csv)
positive_class_probs_3dot0 = create_prob_pos_class_list(probs_3dot0_csv)

In [ ]:
positive_class_probs_0dot2_rawnet = create_prob_pos_class_list(probs_rawnet_0dot2_csv)
positive_class_probs_1dot0_rawnet = create_prob_pos_class_list(probs_rawnet_1dot0_csv)
positive_class_probs_2dot0_rawnet = create_prob_pos_class_list(probs_rawnet_2dot0_csv)
positive_class_probs_3dot0_rawnet = create_prob_pos_class_list(probs_rawnet_3dot0_csv)

In [ ]:
FP_clean, TP_clean, _ = roc_curve(GT_labels, positive_class_probs)
ROC_AUC_clean = auc(FP_clean, TP_clean)

FP_0dot2, TP_0dot2, _ = roc_curve(GT_labels, positive_class_probs_0dot2)
ROC_AUC_0dot2 = auc(FP_0dot2, TP_0dot2)

FP_0dot4, TP_0dot4, _ = roc_curve(GT_labels, positive_class_probs_0dot4)
ROC_AUC_0dot4 = auc(FP_0dot4, TP_0dot4)

FP_0dot6, TP_0dot6, _ = roc_curve(GT_labels, positive_class_probs_0dot6)
ROC_AUC_0dot6 = auc(FP_0dot6, TP_0dot6)

FP_0dot8, TP_0dot8, _ = roc_curve(GT_labels, positive_class_probs_0dot8)
ROC_AUC_0dot8 = auc(FP_0dot8, TP_0dot8)

FP_1dot0, TP_1dot0, _ = roc_curve(GT_labels, positive_class_probs_1dot0)
ROC_AUC_1dot0 = auc(FP_1dot0, TP_1dot0)

FP_2dot0, TP_2dot0, _ = roc_curve(GT_labels, positive_class_probs_2dot0)
ROC_AUC_2dot0 = auc(FP_2dot0, TP_2dot0)

FP_3dot0, TP_3dot0, _ = roc_curve(GT_labels, positive_class_probs_3dot0)
ROC_AUC_3dot0 = auc(FP_3dot0, TP_3dot0)




In [ ]:
FP_0dot2_r, TP_0dot2_r, _ = roc_curve(GT_labels, positive_class_probs_0dot2_rawnet)
ROC_AUC_0dot2_r = auc(FP_0dot2_r, TP_0dot2_r)

FP_1dot0_r, TP_1dot0_r, _ = roc_curve(GT_labels, positive_class_probs_1dot0_rawnet)
ROC_AUC_1dot0_r = auc(FP_1dot0_r, TP_1dot0_r)

FP_2dot0_r, TP_2dot0_r, _ = roc_curve(GT_labels, positive_class_probs_2dot0_rawnet)
ROC_AUC_2dot0_r = auc(FP_2dot0_r, TP_2dot0_r)

FP_3dot0_r, TP_3dot0_r, _ = roc_curve(GT_labels, positive_class_probs_3dot0_rawnet)
ROC_AUC_3dot0_r = auc(FP_3dot0_r, TP_3dot0_r)

In [ ]:
plt.figure()
plt.plot(FP_clean, TP_clean, color='blue', lw=1.5, label=f'Clean (AUC = {ROC_AUC_clean:.2f})')
plt.plot(FP_0dot2, TP_0dot2, color='plum', lw=1.5, label=f'FGSM eps=0.2 (AUC = {ROC_AUC_0dot2:.2f})')
plt.plot(FP_0dot4, TP_0dot4, color='orange', lw=1.5, label=f'FGSM eps=0.4 (AUC = {ROC_AUC_0dot4:.2f})')
plt.plot(FP_0dot6, TP_0dot6, color='gold', lw=1.5, label=f'FGSM eps=0.6 (AUC = {ROC_AUC_0dot6:.2f})')
plt.plot(FP_0dot8, TP_0dot8, color='cyan', lw=1.5, label=f'FGSM eps=0.8 (AUC = {ROC_AUC_0dot8:.2f})')
plt.plot(FP_1dot0, TP_1dot0, color='pink', lw=1.5, label=f'FGSM eps=1.0 (AUC = {ROC_AUC_1dot0:.2f})')
plt.plot(FP_2dot0, TP_2dot0, color='green', lw=1.5, label=f'FGSM eps=2.0 (AUC = {ROC_AUC_2dot0:.2f})')
plt.plot(FP_3dot0, TP_3dot0, color='red', lw=1.5, label=f'FGSM eps=3.0 (AUC = {ROC_AUC_3dot0:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.show()

In [ ]:
plt.figure()
#plt.plot(FP_clean, TP_clean, color='blue', lw=1.5, label=f'Clean (AUC = {ROC_AUC_clean:.2f})')
plt.plot(FP_0dot2_r, TP_0dot2_r, color='plum', lw=1.5, label=f'FGSM eps=0.2 (AUC = {ROC_AUC_0dot2_r:.2f})')
plt.plot(FP_1dot0_r, TP_1dot0_r, color='pink', lw=1.5, label=f'FGSM eps=1.0 (AUC = {ROC_AUC_1dot0_r:.2f})')
plt.plot(FP_2dot0_r, TP_2dot0_r, color='green', lw=1.5, label=f'FGSM eps=2.0 (AUC = {ROC_AUC_2dot0_r:.2f})')
plt.plot(FP_3dot0_r, TP_3dot0_r, color='red', lw=1.5, label=f'FGSM eps=3.0 (AUC = {ROC_AUC_3dot0_r:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve _ RAWNET2')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.show()

In [ ]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d

#https://yangcha.github.io/EER-ROC/

def compute_eer(fpr, tpr):
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    return eer


print(f'EER clean dataset : {compute_eer(FP_clean, TP_clean)*100:.2f}%\n'
      f'EER eps = 0.2 : {compute_eer(FP_0dot2, TP_0dot2)*100:.2f}%\n'
      f'EER eps = 0.4 : {compute_eer(FP_0dot4, TP_0dot4)*100:.2f}%\n'
      f'EER eps = 0.6 : {compute_eer(FP_0dot6, TP_0dot6)*100:.2f}%\n'
      f'EER eps = 0.8 : {compute_eer(FP_0dot8, TP_0dot8)*100:.2f}%\n'
      f'EER eps = 1.0 : {compute_eer(FP_1dot0, TP_1dot0)*100:.2f}%\n'
      f'EER eps = 2.0 : {compute_eer(FP_2dot0, TP_2dot0)*100:.2f}%\n'
      f'EER eps = 3.0 : {compute_eer(FP_3dot0, TP_3dot0)*100:.2f}%')

***
## % of audios with different predictions (wrt clean dataset)

In [ ]:
# def percentage_different(list1, list2):
#     if len(list1) != len(list2):
#         raise ValueError('Lists have different lengths')
#     num_different = sum(1 for x,y in zip(list1, list2) if x!=y)
#     total_elements = len(list1)
#     percentage = (num_different/total_elements)*100
#     return percentage
# 
# print(f'The % of audios with different predictions for eps = 1.0 is {percentage_different(GT_labels, pred_labels_1dot0):.2f}%\n'
#       f'The % of audios with different predictions for eps = 2.0 is {percentage_different(GT_labels, pred_labels_2dot0):.2f}%\n'
#       f'The % of audios with different predictions for eps = 3.0 is {percentage_different(GT_labels, pred_labels_3dot0):.2f}%')

***
## Confusion matrix
* TN true negative, actual class was 0 (BF) and predicted as 0
* FP false positive, actual class was 0, but predicted as 1 (deep fake)
* FN false negative, actual class was 1, but predicted as 0
* TP true positive, actual class was 1 and predicted as 1

In [ ]:
from sklearn.metrics import confusion_matrix

# [ TN    FP
#   FN    TP ]

In [ ]:
cm_clean = confusion_matrix(GT_labels, pred_labels_clean)
print(cm_clean)

In [ ]:
cm_1dot0 = confusion_matrix(GT_labels, pred_labels_1dot0)
print(cm_1dot0)

In [ ]:
cm_2dot0 = confusion_matrix(GT_labels, pred_labels_2dot0)
print(cm_2dot0)

In [ ]:
cm_3dot0 = confusion_matrix(GT_labels, pred_labels_3dot0)
print(cm_3dot0)

In [ ]:
cm_0dot2 = confusion_matrix(GT_labels, pred_labels_0dot2)
cm_0dot4 = confusion_matrix(GT_labels, pred_labels_0dot4)
cm_0dot6 = confusion_matrix(GT_labels, pred_labels_0dot6)
cm_0dot8 = confusion_matrix(GT_labels, pred_labels_0dot8)

In [ ]:
TN = [cm_clean[0,0], cm_0dot2[0,0], cm_0dot4[0,0], cm_0dot6[0,0], cm_0dot8[0,0], cm_1dot0[0,0], cm_2dot0[0,0], cm_3dot0[0,0]]
FP = [cm_clean[0,1], cm_0dot2[0,1], cm_0dot4[0,1], cm_0dot6[0,1], cm_0dot8[0,1], cm_1dot0[0,1], cm_2dot0[0,1], cm_3dot0[0,1]]
FN = [cm_clean[1,0], cm_0dot2[1,0], cm_0dot4[1,0], cm_0dot6[1,0], cm_0dot8[1,0], cm_1dot0[1,0], cm_2dot0[1,0], cm_3dot0[1,0]]
TP = [cm_clean[1,1], cm_0dot2[1,1], cm_0dot4[1,1], cm_0dot6[1,1], cm_0dot8[1,1], cm_1dot0[1,1], cm_2dot0[1,1], cm_3dot0[1,1]]
eps = [0.0, 0.2, 0.4, 0.8, 1.0, 2.0, 3.0]

plt.figure()
plt.plot(eps, TN, color='blue', lw=1.5, label=f'True Negative')
plt.plot(eps, FP, color='red', lw=1.5, label=f'False Positive')
plt.plot(eps, FN, color='green', lw=1.5, label=f'False Negative')
plt.plot(eps, TP, color='orange', lw=1.5, label=f'True Positive')

plt.xlabel('Epsilon')
plt.ylabel('Number of samples')
plt.title('Confusion matrices')
plt.xticks(eps)
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.show()

In [ ]:
TN

***
## Check one single file given the path

In [ ]:
def check_file_given_path(file_path, model_to_use)